In [ ]:
# %%
import osmnx as ox
import pandas as pd
from routing import astar_route

# 1. Load the LA driving graph
G = ox.graph_from_place("Los Angeles, California, USA", network_type="drive")

In [19]:

# ## 2. Load the risk map for a given weather condition
condition = "Rain"  # ← pick one of: Clear, Partially cloudy, Overcast, Rain, Rain, Overcast, Rain, Partially cloudy
csv_path = f"risk_maps/{condition.replace(', ', '_').replace(' ', '_')}.csv"
risk_df = pd.read_csv(csv_path)

# Build a lookup from road_name → risk_score
risk_lookup = dict(zip(risk_df["road_name"], risk_df["risk_score"]))

# ## 3. Annotate every edge with two cost fields
for u, v, key, data in G.edges(keys=True, data=True):
    # base length
    length = data.get("length", 1.0)
    data["cost_distance"] = length

    # determine road name
    name = data.get("name")
    if isinstance(name, list):
        name = name[0]
    if not name:
        name = "Unnamed Road"

    # lookup risk (default = 0)
    risk = risk_lookup.get(name, 0.0)

    # cost_risk = length * (1 + risk) so that risk 0→1 scales cost 1×→2×
    data["cost_risk"] = length * (1 + risk)

# %% [markdown]
# ## 4. Pick your cost attribute based on the condition
# if you want pure distance:
#   cost_attr = "cost_distance"
#
# if you want to penalize risky roads:
#   cost_attr = "cost_risk"

use_risk = True
cost_attr = "cost_risk" if use_risk else "cost_distance"

# ## 5. Run A* and inspect the results
origin = (34.06655181863482, -118.18789017013762)
destination = (34.0647417476385, -118.25823027030046)

node_path, edge_path, road_names = astar_route(
    G,
    origin_point=origin,
    destination_point=destination,
    weight=cost_attr
)
print(node_path)
print(f"Number of nodes in path: {len(node_path)}")
print(f"Number of edges in path: {len(edge_path)}")
print("Roads to follow:", road_names)

[1849409643, 123018936, 123565234, 33155647, 14941504, 1985260166, 2296327020, 1985260196, 281506189, 2258860847, 281506167, 32274961, 72448807, 14798642, 72448746, 17737741, 14941693, 21150896, 16941224, 122977613, 16203358, 122741782, 122977609, 68965696, 1994531073, 250001395, 122656711, 122872202, 122977592, 122977589, 122967648, 122792131, 122977587, 122670717, 268373135, 17677673, 11340695338, 68267399, 122645236, 123637708, 68267394, 1838396767, 60945762, 268653344, 250586509, 268653330, 1842838208, 1838389220, 20400044, 20400214, 60946365, 122663619, 60946368, 122999186, 123008568, 123008570, 122834000, 123008579, 123008582]
Number of nodes in path: 59
Number of edges in path: 58
Roads to follow: ['North Rowan Avenue', 'North Rowan Avenue', 'Valley Boulevard', 'Valley Boulevard', 'Valley Boulevard', 'Valley Boulevard', 'Valley Boulevard', 'North Main Street', 'North Main Street', 'North Main Street', 'North Main Street', 'North Main Street', 'North Main Street', 'North Main Str